In [ ]:
import os, sys
__dir__ = os.path.dirname(os.path.abspath(""))
sys.path.append(__dir__), sys.path.append(os.path.abspath(os.path.join(__dir__, "..")))
from libs import *

from data import ImageDataset
from dassl.modeling.network import *

In [ ]:
train_loaders = {
    "train":torch.utils.data.DataLoader(
        ImageDataset(
            data_dir = "../../datasets/P-ACS/{}/".format("ACS"), 
            augment = True, 
        ), 
        num_workers = 8, batch_size = 16, 
        shuffle = True, 
    ), 
    "val":torch.utils.data.DataLoader(
        ImageDataset(
            data_dir = "../../datasets/P-ACS/{}/".format("P"), 
            augment = False, 
        ), 
        num_workers = 8, batch_size = 16, 
        shuffle = False, 
    ), 
}

In [ ]:
device = torch.device("cuda:1")

Ft, C = torch.load("../../warmup/ckps/P-ACS/P/last.ptl").backbone.to(device), torch.load("../../warmup/ckps/P-ACS/P/last.ptl").classifier.to(device)
for parameter in C.parameters():
    parameter.requires_grad = False
Fs = torchvision.models.resnet18(pretrained = True).to(device)
Fs.fc = nn.Identity()
G = fcn_3x64_gctx().to(device)

In [ ]:
def LFs(
    augmented_logits, logits, 
):
    normalized_augmented_logits, normalized_logits,  = augmented_logits/torch.norm(augmented_logits, p = 2), logits/torch.norm(logits, p = 2), 
    return torch.dist(normalized_augmented_logits, normalized_logits, p = 2)
def LG(
    augmented_logits, logits, 
    m = 0.01, 
    device = torch.device("cpu"), 
):
    normalized_augmented_logits, normalized_logits,  = augmented_logits/torch.norm(augmented_logits, p = 2), logits/torch.norm(logits, p = 2), 
    return torch.minimum(torch.dist(normalized_augmented_logits, normalized_logits, p = 2) - m, torch.zeros(1).to(device))

optimizer_Fs = optim.SGD(
    Fs.parameters(), weight_decay = 5e-4, 
    lr = 5e-4, 
)
scheduler_Fs = optim.lr_scheduler.StepLR(
    optimizer_Fs, 
    step_size = 30, gamma = 0.1, 
)
optimizer_G = optim.SGD(
    G.parameters(), weight_decay = 5e-4, 
    lr = 5e-4, 
)
scheduler_G = optim.lr_scheduler.StepLR(
    optimizer_G, 
    step_size = 30, gamma = 0.1, 
)

In [ ]:
num_epochs = 60
for epoch in range(1, num_epochs + 1):
    print("epoch {}/{}".format(epoch, num_epochs) + "\n" + " - "*16)

    for images, labels in tqdm.tqdm(train_loaders["train"]):
        images, labels = images.to(device), labels.to(device)

        augmented_images = G(images)
        augmented_logits, logits,  = Fs(augmented_images.float()), Ft(images.float()), 
        break
    break